In [ ]:
from homog_pseudospectral import *
from sw2d import *
from clawpack import pyclaw
font = {'size'   : 15}
matplotlib.rc('font', **font)

## Comparison of 3 solutions over sinusoidal bathymetry

To generate the data for this, run:

`mpirun -np 8 python3 sw2d.py cells_per_period=160 bjump=0.3 btype=sinusoidal channel_width=1.0 outdir="./_b03_sin_yw1_160cpp_g98_a05_t300_L200" num_output_times=300 solver_type=classic tfinal=300 L=200 pulse_amplitude=0.05`

You also need to run the MATLAB code.  Open MATLAB, navigate to the `matlab` directory under this one, and run the command `sw2d`.

First we solve the 1D homogenized equations:

In [ ]:
# This takes a few minutes to run
m = 32000
T = 300
x, xi, momentum, surface_homog, anim = solve_SWH(h_amp=0.05,m=m,tmax=T,btype="sinusoidal",b_amp=0.3,num_plots=T,L=2000,make_anim=False)

In [ ]:
tmax = T
fig, ax = plt.subplots(2,2,figsize=(12,8),sharey=True)
plt.subplots_adjust(wspace=0.02)
ytext = 0.03
props = dict(facecolor='lightgrey', alpha=0.2)
t_int = tmax/(len(surface_homog)-1)
lw = 2
PS_path="./matlab/32k/"#/sept15-run/"
FV_path = "./_b03_sin_yw1_160cpp_g98_a05_t300_L200"

def load_frame(i):
    surface_ps = np.loadtxt(PS_path+"eta_"+str(i)+".txt")
    xc_ps = np.loadtxt(PS_path+"x32.txt")
    frame = pyclaw.Solution(i,file_format='petsc',file_prefix='claw',read_aux=True,path=FV_path)
    n = frame.q.shape[2]
    b = frame.aux[0,:,:]
    h = frame.q[0,:,:]
    surface_fv = h+b; surface_fv = surface_fv.mean(1)
    xc_fv = frame.state.grid.x.centers
    return xc_ps, surface_ps, xc_fv, surface_fv

def add_plot(frame,axis,xmin,xmax,offset=0):
    i = frame
    axis = axis
    xc_ps, surface_ps, xc_fv, surface_fv = load_frame(i)
    axis.plot(xc_fv,surface_fv,"-k",label="Variable-bathymetry solution (FV averaged)",lw=lw)
    axis.plot(xc_fv+100,surface_fv,"-k",lw=lw)
    axis.plot(xc_fv+200,surface_fv,"-k",lw=lw)
    axis.plot(xc_fv+300,surface_fv,"-k",lw=lw)
    axis.plot(xc_fv+400,surface_fv,"-k",lw=lw)
    axis.plot(xc_fv+600,surface_fv,"-k",lw=lw)
    axis.plot(xc_ps+offset,surface_ps,"--",label="Variable-bathymetry solution (PS averaged)",lw=lw)
    axis.plot(x,surface_homog[i],"--r",label="Homogenized solution",lw=lw)
    axis.set_xlim(xmin,xmax); axis.set_ylim(-1e-3,0.05)
    axis.set_xticks(range(int(xmin)+5, int(xmax)-4, 10))
    axis.text(xmin+(xmax-xmin)/10,ytext,'$t=%3.1f$' % (i*t_int),bbox=props)

add_plot(50,axis=ax[0,0],xmin=140,xmax=180,offset=0)
fig.legend(loc='upper center', fancybox=True, shadow=True);
add_plot(100,axis=ax[0,1],xmin=295,xmax=335,offset=0)
add_plot(150,axis=ax[1,0],xmin=450,xmax=490,offset=0)
add_plot(200,axis=ax[1,1],xmin=610,xmax=650,offset=0)

#add_plot(30,axis=ax[0,0],xmin=70,xmax=110,offset=0)
#fig.legend(loc='upper center', fancybox=True, shadow=True);
#add_plot(70,axis=ax[0,1],xmin=200,xmax=240,offset=0)
#add_plot(120,axis=ax[1,0],xmin=360,xmax=400,offset=0)
#add_plot(150,axis=ax[1,1],xmin=450,xmax=490,offset=0)

fig.text(0.5, 0.04, 'x', ha='center', va='center')
fig.text(0.06, 0.5, r'Surface elevation ($\eta$)', ha='center', va='center', rotation='vertical')
plt.savefig('comparison_sinusoidal_4panel.pdf')

### Comparison of solution differences over time

Here we compute the differences between the FV and PS solutions versus the homogenized solution and how the differences grow over time.
It's a bit tricky to match up the solutions, especially because the FV code is run on a domain of length 100 with periodic boundary conditions, in order to reduce the computational cost.  So we only compute the error over a window containing the advancing solitary wave train, and we avoid times when that train overlaps with the boundary.

In [ ]:
difference_FV = []
difference_PS = []
tt = np.array([5, 10, 15, 20, 25, 40, 45, 50, 55, 70,75,80,85,90,105,110,115,135,140,
          150,165,170,175,180,185,200,205,210,215,230,235,240,245,265,270,275,295,300])
for i in tt:
    offset = get_offset(i)
    xc_ps, surface_ps, xc_fv, surface_fv = load_frame(int(i))
    dx_ps = xc_ps[1]-xc_ps[0]
    dx_fv = xc_fv[1] - xc_fv[0]
    dx_homog = x[1]-x[0]
    surface_homog_val = surface_homog[i][16000+offset*1600:16000+(offset+1)*1600]
    surface_ps_val = surface_ps[16000+offset*1600:16000+(offset+1)*1600]
    difference_FV.append(np.linalg.norm(surface_fv[::10]-surface_homog_val[:],1)*dx_ps)
    difference_PS.append(np.linalg.norm(surface_ps_val-surface_homog_val[:],1)*dx_ps)

In [ ]:
plt.plot(tt,difference_FV)
plt.plot(tt,difference_PS)
plt.legend(["FV","PS"])

## Comparison of solutions over discontinuous bathymetry

For this comparison we omit the 2D pseudospectral solver since it can't deal with non-smooth bathymetry.

In [ ]:
b_amp = 1.2

m = 8192*4
T = 400
x, xi, momentum, eta, anim = solve_SWH(h_amp=0.05,m=m,tmax=T,b_amp=b_amp,num_plots=800,L=2800,make_anim=False)

To generate the FV simulation data, run

`mpirun -np 8 python3 sw2d.py cells_per_period=40 bjump=1.2 btype=pwc channel_width=1.0 outdir="./_b12_pwc_yw1_40cpp_g98_a05_t400_L400_cl" num_output_times=800 solver_type=classic tfinal=400 L=400`

In [ ]:
tmax = T
fig, ax = plt.subplots(2,2,figsize=(12,8),sharey=True)
plt.subplots_adjust(wspace=0.02)
ytext = 0.03
props = dict(facecolor='lightgrey', alpha=0.2)
t_int = tmax/(len(eta)-1)
lw = 2
path="./_b12_pwc_yw1_40cpp_g98_a05_t400_L400_cl"

def load_frame(i):
    frame = pyclaw.Solution(i,file_format='petsc',file_prefix='claw',read_aux=True,path=path)
    n = frame.q.shape[2]
    b = frame.aux[0,:,:]
    h = frame.q[0,:,:]
    surface = h+b
    surface = surface.mean(axis=1)
    xc = frame.state.grid.x.centers

    return xc, surface

def add_plot(frame,axis,xmin,xmax,offset=0):
    i = frame
    axis = axis
    #xfine, pfine = fine_resolution(p_ps[i],10000,x_ps,xi_ps)
    #axis.set_ylim(0.99,1.17)
    xc, surface = load_frame(i)
    axis.plot(xc+offset,surface[:],"-k",label="Variable-bathymetry solution (averaged)",lw=lw)
    axis.plot(x,eta[i],"--",label="Homogenized solution",lw=lw)
    #axis.plot(xfine,pfine,'-.',lw=lw, label='Homogenized p-system')
    axis.set_xlim(xmin,xmax)
    axis.set_xticks(range(int(xmin)+5, int(xmax)-4, 10))
    #axis.set_yticks([1, 1.05, 1.1])  # set only 3 y-axis tick labels
    axis.text(xmin+(xmax-xmin)/10,ytext,'$t=%3.1f$' % (i*t_int),bbox=props)

add_plot(100,axis=ax[0,0],xmin=135,xmax=175,offset=0)
fig.legend(loc='upper center', fancybox=True, shadow=True);
add_plot(200,axis=ax[0,1],xmin=300,xmax=340,offset=200)
add_plot(400,axis=ax[1,0],xmin=610,xmax=650,offset=600)
add_plot(800,axis=ax[1,1],xmin=1240,xmax=1290,offset=1200)

fig.text(0.5, 0.04, 'x', ha='center', va='center')
fig.text(0.06, 0.5, r'Surface elevation ($\eta$)', ha='center', va='center', rotation='vertical')
plt.savefig('comparison.pdf')

# Solitary wave shape investigation

## Train of waves

The data for the next figures is generated via

`mpirun -np 8 python3 sw2d.py cells_per_period=40 bjump=1.2 btype=pwc channel_width=1.0 outdir="./_b12_pwc_yw1_40cpp_g98_a08_t400_L200_cl" num_output_times=1400 solver_type=classic tfinal=1400 L=200`

In [ ]:
i = 399
path = "_b12_pwc_yw1_40cpp_g98_a08_t400_L200_cl"
frame = pyclaw.Solution(i,file_format='petsc',file_prefix='claw',read_aux=True,path=path)
b = frame.aux[0,:,:]
n = frame.q.shape[2]
h = frame.q[0,:,:]
print(h.shape)
surface = h+b
surface_mean = surface.mean(axis=1)
xc = frame.state.grid.x.centers
yc = frame.state.grid.y.centers

offset = 2500
plt.figure(figsize=(12,4))
plt.plot(xc,surface_mean)
#plt.xlim(480,520)
#plt.legend(['Direct','Homogenized'])

delta = np.diff(surface_mean)
dd = delta[1:]*delta[:-1] * (surface_mean[2:]>0.0011)
peaks = np.where(dd<0)[0]

peaks = peaks[2:]

#plt.plot(xc[peaks],surface_mean[peaks],'ok');
print(peaks)
plt.xlim(0,100)
plt.ylabel(r"$\overline{\eta}$")
plt.xlabel(r"x");
plt.savefig("three_sol.pdf", bbox_inches="tight")

## sech^2 fit

In [ ]:
centers = xc[peaks]
amplitudes = surface_mean[peaks]
lw = 2
i = 0
for c, a in zip(centers,amplitudes):
    plt.plot((xc-c-0.03)*np.sqrt(a),surface_mean/a,lw=lw,label=str(i))
    i += 1
#plt.plot([0,0],[0,1],"--k")
plt.ylim(-0.1,1.1)
plt.xlim(-0.5,0.5);

xx = np.linspace(-1,1,1000)

fac = 4.85
sech2 = 1./(np.cosh(fac*(xx)))**2
plt.plot(xx,sech2,"--k")
#plt.legend();
plt.ylabel(r"$\overline{\eta}$")
plt.xlabel(r"$\hat{x}$");
plt.savefig("rescaled.pdf", bbox_inches="tight")

`mpirun -np 8 python3 sw2d.py cells_per_period=40 bjump=1.2 btype=pwc channel_width=1.0 outdir="./_b12_pwc_yw1_40cpp_g98_a05_t400_L200_cl" num_output_times=600 solver_type=classic tfinal=600 L=200`

Note that for this one we must use the default minmod limiter; with MC there is an instability at late times.

In [ ]:
from diffracton_shape import *

n = 520
path="./_b12_pwc_yw1_40cpp_g98_a05_t400_L200_cl"
frame = pyclaw.Solution(n,file_format='petsc',file_prefix='claw',read_aux=True,path=path)
n = frame.q.shape[2]
b = frame.aux[0,:,:]
h = frame.q[0,:,:]
surface = h+b
surface_mean = surface.mean(axis=1)
xc = frame.state.grid.x.centers
yc = frame.state.grid.y.centers

offset = 2500

delta = np.diff(surface_mean)
dd = delta[1:]*delta[:-1] * (surface_mean[2:]>0.0011)
peaks = np.where(dd<0)[0]

i = 3
X,Y = frame.state.grid.p_centers
Sh, Shu, Shv = initialize_solitary_wave(A=surface_mean[peaks[i]],X=X-xc[peaks[i]],Y=Y,fac=4.85,b=b)
Ssurface = Sh+b
Ssurface_mean = Ssurface.mean(axis=1)

for iy in [10, 30]:
    plt.plot(xc,surface[:,iy],'k')
    plt.plot(xc+0.01,Ssurface[:,iy],'--')
    plt.xlim(62,68)
    #plt.xlim(66,75)
    plt.xlabel("$x$")
    plt.ylabel(r"$\eta(x,y)$");

#plt.legend(["$y=-19/80$","$y=21/80$"],fontsize=10);
plt.savefig("y_slices_comparison.pdf",bbox_inches="tight");

In [ ]:
hv = frame.q[2,:,:]
hv_mean = hv.mean(axis=1)
Shv_mean = Shv.mean(axis=1)

for iy in [15, 35]:
    plt.plot(xc,hv[:,iy],"k")
    plt.plot(xc+0.03,Shv[:,iy],"--")
    plt.xlim(60,70)
    plt.xlabel("$x$")
    plt.ylabel(r"$p(x,y)$");

#plt.legend(["$y=-19/80$","$y=21/80$"],fontsize=10);
plt.savefig("y_slices_comparison_p.pdf",bbox_inches="tight");